<a href="https://colab.research.google.com/github/jbliaw/NBA-Reference/blob/main/Basketball_Reference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
%load_ext watermark

In [14]:
watermark

Last updated: 2022-08-09T04:52:02.154840+00:00

Python implementation: CPython
Python version       : 3.7.13
IPython version      : 5.5.0

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.4.188+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



## Preparation

In [2]:
# Necessary imports
from bs4 import BeautifulSoup as bs
from bs4 import Comment
import requests
import re
import pandas as pd

In [3]:
# Function to get individual team stats
def get_team_stats(team): 
  listy = []
  stats = {}

  for index, row in enumerate(team): 
    row.replace("'","")
    if row.startswith("data-stat"):
      row.startswith("data-stat")
      listy.append(row)
    for index, row in enumerate(listy):
      if index == 0 or index == 20:
        continue
      stat = re.findall(r'"(.*?)"', row)[0]
      value = re.findall(r'>(.+?)<', row)[0]
      stats['{}'.format(stat)] = value

  return stats

In [4]:
# Function to get both team stats into dataframe
def get_games_info(url): 

  r = requests.get(url)
  soup = bs(r.content)
  feet = soup.find_all('tfoot')
  first_team = str(feet[0].encode('utf-8'))
  second_team = str(feet[8].encode('utf-8'))
  first_team = first_team.split()
  second_team = second_team.split()

  stats1 = get_team_stats(first_team)
  stats2 = get_team_stats(second_team)

  df = pd.DataFrame([stats1, stats2])

  return df

In [5]:
# Function to get team names
def get_team_names(teams): 
  pre_team_names = []
  team_names = []

  for index, item in enumerate(teams):
    stringy = str(item)
    name_list = stringy.split('"',)
    pre_team_names.append(name_list[3])

  for team in pre_team_names: 
    home = team[0:3]
    visitor = team[13:16]
    team_names.append(home)
    team_names.append(visitor)

  return team_names

## Get Your Data Here

In [6]:
# Enter url of site
url = 'https://www.basketball-reference.com/leagues/NBA_2019_games-february.html'

In [7]:
# Set up and grab initial data from main page
r = requests.get(url)
soup = bs(r.content)
box_scores = soup.find_all('td', {'data-stat': 'box_score_text'})
teams = soup.find_all('td', {'data-stat': 'visitor_team_name'})
team_names = get_team_names(teams)

In [8]:
# Grab box score data from each box score site
# NOTE: This will take a while. This step will print the index at which it is working. 
base_path = "https://www.basketball-reference.com/"

games_info_list = []

for index, game in enumerate(box_scores):
  relative_path = game.a['href']
  full_path = base_path + relative_path

  if index % 10 == 0: 
    print(index)
  
  games_info_list.append(get_games_info(full_path))

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150


In [9]:
# Create into dataframe
df1 = pd.DataFrame({'Teams': team_names})
df2 = pd.concat(games_info_list)

df1.reset_index(drop=True, inplace=True)
df2.reset_index(drop=True, inplace=True)
df = pd.concat([df1, df2], axis=1)

df.drop(df.iloc[:, 20:35], inplace=True, axis=1)
df = df.set_index('Teams')

In [10]:
# Ta-Da!
print(df.shape)
df

(316, 19)


,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,orb,drb,trb,ast,stl,blk,tov,pf,pts
Teams,,,,,,,,,,,,,,,,,,,
MEM,240,35,80,.438,9,29,.310,13,17,.765,9,31,40,23,8,3,11,22,92
CHO,240,32,73,.438,12,29,.414,24,28,.857,6,34,40,15,6,7,12,17,100
BOS,240,46,100,.460,13,37,.351,8,10,.800,11,32,43,25,11,7,7,18,113
NYK,240,39,86,.453,9,31,.290,12,12,1.000,8,40,48,21,2,2,17,15,99
OKC,240,44,91,.484,16,30,.533,14,19,.737,13,37,50,23,10,4,15,20,118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOU,240,38,77,.494,19,46,.413,26,28,.929,8,28,36,22,6,5,15,21,121
PHI,240,42,91,.462,12,30,.400,12,19,.632,8,36,44,33,9,5,13,24,108
OKC,240,38,87,.437,8,27,.296,20,28,.714,8,36,44,28,5,3,12,18,104


In [ ]:
# Save to Excel (feel free to change month name accordingly)
df.to_excel('february_stats.xlsx')